ref: https://www.kaggle.com/hamishdickson/bidirectional-lstm-in-keras-with-glove-embeddings

# LOAD libs

In [1]:
import pandas as pd
import numpy as np
import time
import os, gc
from tqdm import tqdm

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

import re

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import torch
from torch import nn, cuda
from torch.nn import functional as F
from torch.utils.data import TensorDataset, Subset, DataLoader
from torch.optim import Adam, Optimizer
from torch.optim.lr_scheduler import _LRScheduler, LambdaLR, ReduceLROnPlateau

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from sklearn.metrics import roc_auc_score, f1_score

Using TensorFlow backend.


In [3]:
use_cuda = cuda.is_available()
use_cuda

True

In [4]:
def seed_everything(seed=123):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [5]:
class SequenceBucketCollator():
    def __init__(self, choose_length, sequence_index, length_index, label_index=None):
        self.choose_length = choose_length
        self.sequence_index = sequence_index
        self.length_index = length_index
        self.label_index = label_index
        
    def __call__(self, batch):
        batch = [torch.stack(x) for x in list(zip(*batch))]
        
        sequences = batch[self.sequence_index]
        lengths = batch[self.length_index]
        
        length = self.choose_length(lengths)
        mask = torch.arange(start=maxlen, end=0, step=-1) < length
        padded_sequences = sequences[:, mask]
        
        batch[self.sequence_index] = padded_sequences
        
        if self.label_index is not None:
            return [x for i, x in enumerate(batch) if i != self.label_index], batch[self.label_index]
    
        return batch

In [6]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

class NeuralNet(nn.Module):

    def __init__(self, embedding_matrix):
        super(NeuralNet, self).__init__()

        lstm_hidden_size = 120
        gru_hidden_size = 60
        self.gru_hidden_size = gru_hidden_size

        self.embedding = nn.Embedding(*embedding_matrix.shape)
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = False
        self.embedding_dropout = nn.Dropout2d(0.2)

        self.lstm = nn.LSTM(embedding_matrix.shape[1], lstm_hidden_size, bidirectional=True, batch_first=True)
        self.gru = nn.GRU(lstm_hidden_size * 2, gru_hidden_size, bidirectional=True, batch_first=True)

        self.linear = nn.Linear(gru_hidden_size * 6, 20)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.1)
        self.out = nn.Linear(20, 1)
        
    def apply_spatial_dropout(self, h_embedding):
        h_embedding = h_embedding.transpose(1, 2).unsqueeze(2)
        h_embedding = self.embedding_dropout(h_embedding).squeeze(2).transpose(1, 2)
        return h_embedding

    def forward(self, x, normal_feats, lengths=None):
        h_embedding = self.embedding(x.long())
        h_embedding = self.apply_spatial_dropout(h_embedding)

        h_lstm, _ = self.lstm(h_embedding)
        h_gru, hh_gru = self.gru(h_lstm)

        hh_gru = hh_gru.view(-1, self.gru_hidden_size * 2)

        avg_pool = torch.mean(h_gru, 1)
        max_pool, _ = torch.max(h_gru, 1)

#         normal_linear  = F.relu(self.normal_linear(normal_feats.float()))

        conc = torch.cat((hh_gru, avg_pool, max_pool), 1)
        conc = self.relu(self.linear(conc))
        conc = self.dropout(conc)
        out = self.out(conc)

        return out


In [7]:
def train_model(n_epochs=4, accumulation_step=2, **kwargs):
    
    optimizer = Adam(model.parameters(), lr=0.001)
    scheduler = LambdaLR(optimizer, lambda epoch: 0.6 ** epoch)
    checkpoint_weights = [2 ** epoch for epoch in range(n_epochs)]
    
    best_epoch = -1
    best_valid_score = 0.
    best_valid_loss = 1.
    all_train_loss = []
    all_valid_loss = []
    total_preds = []
    
    for epoch in range(n_epochs):
        
        start_time = time.time()

        train_loss = train_one_epoch(model, criterion, train_loader, optimizer, accumulation_step)
        val_loss, val_score = validation(model, criterion, valid_loader)
    
#         if val_score > best_valid_score:
#             best_valid_score = val_score
#             torch.save(model.state_dict(), 'best_score{}.pt'.format(fold))
    
        elapsed = time.time() - start_time
        
        lr = [_['lr'] for _ in optimizer.param_groups]
        print("Epoch {} - train_loss: {:.6f}  val_loss: {:.6f}  val_score: {:.6f}  lr: {:.5f}  time: {:.0f}s".format(
                epoch+1, train_loss, val_loss, val_score, lr[0], elapsed))

        # inference
        test_preds = inference_test(model, test_loader)
        total_preds.append(test_preds)
        
        # scheduler update
        scheduler.step()
    
    total_preds = np.average(total_preds, weights=checkpoint_weights, axis=0)

    return total_preds, val_score, val_loss

In [8]:
def inference_test(model, test_loader):
    model.eval()

    test_preds = np.zeros((len(test_dataset), 1))

    with torch.no_grad():
        for i, inputs in enumerate(test_loader):
            if use_cuda:
                inputs[0] = inputs[0].cuda()
                inputs[1] = inputs[1].cuda()
#                 inputs[2] = inputs[2].cuda()

            outputs = model(inputs[0], inputs[1])
#             outputs = model(inputs[0], inputs[1], inputs[2])
            test_preds[i * batch_size:(i+1) * batch_size] = sigmoid(outputs.cpu().numpy())
    
    return test_preds


In [9]:
def train_one_epoch(model, criterion, train_loader, optimizer, accumulation_step=2):
    
    model.train()
    train_loss = 0.
    
    optimizer.zero_grad()

#     for i, (inputs, targets) in tqdm(enumerate(train_loader), desc='train', total=len(train_loader)):
    for i, (inputs, targets) in enumerate(train_loader):

        if use_cuda:
            inputs[0] = inputs[0].cuda()
            inputs[1] = inputs[1].cuda()
#             inputs[2] = inputs[2].cuda()
            targets = targets.cuda()    
            
        preds = model(inputs[0], inputs[1])
        preds = model(inputs[0], inputs[1])
        loss = criterion(preds, targets)

        loss.backward()
        
        if accumulation_step:
            if (i+1) % accumulation_step == 0:  
                optimizer.step()
                optimizer.zero_grad()
        else:
            optimizer.step()
            optimizer.zero_grad()

        train_loss += loss.item() / len(train_loader)
        
    return train_loss


def validation(model, criterion, valid_loader):
    
    model.eval()
    valid_preds = np.zeros((len(valid_dataset), 1))
    valid_targets = np.zeros((len(valid_dataset), 1))
    val_loss = 0.
    
    with torch.no_grad():
#         for i, (inputs, targets) in tqdm(enumerate(valid_loader), desc='valid', total=len(valid_loader)):
        for i, (inputs, targets) in enumerate(valid_loader):
            
            valid_targets[i * batch_size: (i+1) * batch_size] = targets.numpy().copy()
            
            if use_cuda:
                inputs[0] = inputs[0].cuda()
                inputs[1] = inputs[1].cuda()
#                 inputs[2] = inputs[2].cuda()
                targets = targets.cuda()   
            
            outputs = model(inputs[0], inputs[1])
#             outputs = model(inputs[0], inputs[1], inputs[2])
            loss = criterion(outputs, targets)
            
            valid_preds[i * batch_size: (i+1) * batch_size] = sigmoid(outputs.detach().cpu().numpy())
            
            val_loss += loss.item() / len(valid_loader)
    
    val_score = roc_auc_score(valid_targets, valid_preds)
#     valid_preds = np.where(valid_preds>=0.1, 1, 0)
#     val_score = f1_score(valid_targets, valid_preds)
    
    
    return val_loss, val_score

In [10]:
%%time
train_df=pd.read_csv("../KB_NLP/morphs/komo_morphs_train.csv")
test_df=pd.read_csv("../KB_NLP/morphs/komo_morphs_test.csv")

CPU times: user 1.94 s, sys: 92.1 ms, total: 2.03 s
Wall time: 2.03 s


In [11]:
train_df.head()

,id,year_month,text,smishing,komo_morphs
0,0,2017-01,XXX은행성산XXX팀장입니다.행복한주말되세요,0,XXX 은행 성산 XXX 팀장 이 ㅂ니다 . 행복 한 주말 되 시 어요
1,1,2017-01,오늘도많이웃으시는하루시작하세요XXX은행 진월동VIP라운지 XXX올림,0,오늘 도 많이 웃 으시 는 하루 시작 하 시 어요 XXX 은행 지 ㄴ 월동 VIP ...
2,2,2017-01,안녕하십니까 고객님. XXX은행입니다.금일 납부하셔야 할 금액은 153600원 입니...,0,안녕하십니까 고객 님 . XXX 은행 이 ㅂ니다 . 금일 납부 하 시 어야 하 ㄹ ...
3,4,2017-01,XXX 고객님안녕하세요XXX은행 XXX지점입니다지난 한 해 동안 저희 XXX지점에 ...,0,XXX 고객 님 안녕하세요 XXX 은행 XXX 지점 이 ㅂ니다 지나 ㄴ 한 하 아 ...
4,5,2017-01,1월은 새로움이 가득XXX입니다.올 한해 더 많이행복한 한해되시길바랍니다,0,1월 은 새롭 ㅁ 이 가득 XXX 이 ㅂ니다 . 올 한하 아 더 많이 행복 하 ㄴ ...


In [12]:
pd.set_option('display.max_colwidth',-1)
train_df.head(10)

,id,year_month,text,smishing,komo_morphs
0,0,2017-01,XXX은행성산XXX팀장입니다.행복한주말되세요,0,XXX 은행 성산 XXX 팀장 이 ㅂ니다 . 행복 한 주말 되 시 어요
1,1,2017-01,오늘도많이웃으시는하루시작하세요XXX은행 진월동VIP라운지 XXX올림,0,오늘 도 많이 웃 으시 는 하루 시작 하 시 어요 XXX 은행 지 ㄴ 월동 VIP 라운지 XXX 올림
2,2,2017-01,안녕하십니까 고객님. XXX은행입니다.금일 납부하셔야 할 금액은 153600원 입니다.감사합니다. 새해 복 많이 받으십시오.XXX은행옥포XXX올림,0,안녕하십니까 고객 님 . XXX 은행 이 ㅂ니다 . 금일 납부 하 시 어야 하 ㄹ 금액 은 153600 원 이 ㅂ니다 . 감사 하 ㅂ니다 . 새해 복 많이 받 으시 ㅂ시오 . XXX 은행 옥포 XXX 올림
3,4,2017-01,XXX 고객님안녕하세요XXX은행 XXX지점입니다지난 한 해 동안 저희 XXX지점에 보내주신 성원에 감사드립니다. 설렘으로 시작한 2017년소망하시는 일 모두 이XXX 고객님의 가정에 늘 건강과 행복이 함께하길 기원하겠습니다. 사랑하는 가족과 함께 정을 나누는 행복한 설 명절 보내세요 XXX은행 XXX지점직원일동,0,XXX 고객 님 안녕하세요 XXX 은행 XXX 지점 이 ㅂ니다 지나 ㄴ 한 하 아 동안 저희 XXX 지점 에 보내 어 주 시 ㄴ 성원 에 감사 드리 ㅂ니다 . 설레 ㅁ 으로 시작 하 ㄴ 2017년 소망 하 시 는 일 모두 이 XXX 고객 님 의 가정 에 늘 건강 과 행복 이 함께 하 기 ㄹ 기원 하 겠 습니다 . 사랑 하 는 가족 과 함께 정 을 나누 는 행복 하 ㄴ 설 명절 보내 시 어요 XXX 은행 XXX 지점 직원 일동
4,5,2017-01,1월은 새로움이 가득XXX입니다.올 한해 더 많이행복한 한해되시길바랍니다,0,1월 은 새롭 ㅁ 이 가득 XXX 이 ㅂ니다 . 올 한하 아 더 많이 행복 하 ㄴ 한해 되 시 기 ㄹ 바라 ㅂ니다
5,6,2017-01,행복한주말보내세요XXX용현남전담직원대리 XXX올림,0,행복 하 ㄴ 주말 보 내세 요 XXX 용 현 남 전담 직원 대리 XXX 올림
6,7,2017-01,XXX 고객님 안녕하세요XXX은행 무교지점 XXX과장입니다 오늘 아침에 눈을 뜨니 눈이 많이 와서 온 세상이 하얗게 변했더군요어릴적에는 눈이 마냥 좋기만 했는데눈을 보는 순간 좋기보다는 먼저. 출근을 걱정하게 되더라구요이제 진정한 어른이 되었음을 느끼게 됩니다새해 인사를 드리기전에..제가 발령이 나서 본점으로 가게되었어요ㅠㅠ. 진정한 구정 새해인사보다 이별인사를 먼저 드리게 되어 송구할 뿐입니다그동안 보내주신 성원에 감사.또 감사드리며XXX은행에 있는XXX고객님을 평생 저의 고객으로 모시겠습니다.새해 복 많이 받으시고 2017년 원하고 바라시는 모든일 이루시길 기원합니다XXX은행 무교지점 XXX과장올림,0,XXX 고객 님 안녕하세요 XXX 은행 무교 지점 XXX 과장 이 ㅂ니다 오늘 아침 에 눈 을 뜨 니 눈 이 많이 오 아서 오 ㄴ 세상 이 하얗 게 변하 았 더군요 어리 ㄹ 적 에 는 눈 이 마냥 좋 기 만 하 았 는데 눈 을 보 는 순간 좋 기 보다 는 먼저 . 출근 을 걱정 하 게 되 더라구요 이제 진정 하 ㄴ 어른 이 되 었 음 을 느끼 게 되 ㅂ니다 새해 인사 를 드리 기 전 에 .. 제가 발령 이 나서 어 본점 으로 가게되었어요ㅠㅠ. 진정 하 ㄴ 구정 새 해인사 보다 이별 인사 를 먼저 드리 게 되 어 송구 하 ㄹ 뿐 이 ㅂ니다 그동안 보내 어 주 시 ㄴ 성원 에 감사 . 또 감사 드리 며 XXX 은행 에 있 는 XXX 고객 님 을 평생 저 의 고객 으로 모시 겠 습니다 . 새해 복 많이 받 으시 고 2017 년 원하 고 바라 시 는 모든 일 이 루시 길 기원 하 ㅂ니다 XXX 은행 무교 지점 XXX 과 장 올림
7,8,2017-01,XXX 고객님지난 한해 베풀어 주신 은혜 진심으로 감사 드립니다.가슴 깊이 간직 하겠습니다.정유년 새해 가족 모두 행복하시고 뜻 하는 바 모두이루시길 진심으로 소망합니다XXX XXX은행 XXX 지점장 정 XXX 배상,0,XXX 고객 님 지나 ㄴ 한하 아 베풀 어 주 시 ㄴ 은혜 진심 으로 감사 드리 ㅂ니다 . 가슴 깊이 간직 하 겠 습니다 . 정유 년 새해 가족 모두 행복 하 시 고 뜻 하 는 바 모두 이루 시 기 ㄹ 진심 으로 소망 하 ㅂ니다 XXX XXX 은행 XXX 지점장 정 XXX 배상
8,9,2017-01,설연휴 가족들과 훈훈한 정 나누시고 정겨운추억 많이 만드세요XXX오XXX올림,0,설 연휴 가족 들 과 훈훈 하 ㄴ 정 나누 시 고 정겨운 추억 많이 만들 시 어요 XXX 오 XXX 올림
9,10,2017-01,(광고)XXXBaXXX고객님들 뒤엔XXX 언제나 XXX새로운 마음가짐으로 새롭게 준비합니다.당행상품의 자격기준과 심사기준이 완화되어 상품에 대해 간단하게 상품정보 전달드립니다.수신을 희망하지 않으실 경우에는 거부 라는 답장을 주시면 KISA 수신거부 목록에 등록을 시켜 두번 다시 발송되지 않도록 조치를 취해드리겠습니다.신청방법: XXX-XXX-XXX 터치하셔서 문자로 상담 또는 00시 00분 상담이라고 답장을 보내주시거나 전화를 주시면 친절하고 안전한 상담으로 도움드리겠습니다.카카오톡 상담서비스 시행중 citibankloan친구추가 후 상담신청이런 분들께 권해드립니다.1. 시중은행권에서 기존한도를 모두 사용 중이신 분.2. 기대출의 원리금 균등방식으로 인해 월 불입금액이 부담스러우신 분.3. 총 채무금액은 작으나 채무건수가 많아서 관리가 안 되시는 XXX고객님들께 말씀드립니다. 높은 금리 때문에 힘들다고 느끼셨을 때 대출이 필요하실 때 언제나XXX 고객님들 뒤에서 든든한 파트너로서 도와드리겠습니다.당행 상품특징1. 당행 거래가 없으셔도 진행가능.2. 시중은행권보다 높은 한도.3. 자체 등급으로 판단하므로 낮은신용자도 진행가능.4. 부채가 많다면 부채 통합상품으로 전환가능.5. 원리금 균등방식을 이자만 납입하는 만기일시로 전환가능.6. 가상 조회를 통해 한도 및 금리를 한 번에 확인가능.당행 상품안내1. 직장인전용 신용상품한도: 최대 1억 4000만 원까지.금리: 최저 2.97%자격: XXX은행 자체 등급과 기업 리스트에 따라 차등적용.2. 채무통합 전환상품한도: 연봉 대비 300%까지.금리: 최저 2.97%자격: XXX은행 자체 등급과 기업 리스트에 따라 차등적용.3. 새 희망 홀씨한도: 최대 2500만 원.금리: 8.68%12%자격: 연봉 4000만 원 이하 고객 대상으로 XXX 자체 등급에 따라(광고)XXX,1,( 광고 ) XXXBaXXX 고객 님 들 뒤 엔 XXX 언제나 XXX 새롭 ㄴ 마음가짐 으로 새롭 게 준비 하 ㅂ니다 . 당행 상품 의 자격 기준 과 심사 기준 이 완화 되 어 상품 에 대하 아 간단 하 게 상품 정보 전달 드리 ㅂ니다 . 수신 을 희망 하 지 않 으시 ㄹ 경우 에 는 거부 이 라는 답장 을 주 시 면 KISA 수신 거부 목록 에 등록 을 시키 어 두 번 다시 발송 되 지 않 도록 조치 를 취하 아 드리 겠 습니다 . 신청 방법 : XXX - XXX - XXX 터치 하 시 어서 문자 로 상담 또는 00 시 00 분 상담 이 라고 답장 을 보내 어 주 시 거나 전화 를 주 시 면 친절 하 고 안전 하 ㄴ 상담 으로 도움 드리 겠 습니다 . 카카오톡 상담 서비스 시행 중 citibankloan 친구 추가 후 상담 신청 이런 분 들 께 권하 아 드리 ㅂ니다 . 1 . 시중 은행 권 에서 기존 한도 를 모두 사용 중 이신 분 . 2 . 기 대출 의 원리금 균등 방식 으로 인해 월 불 입금액 이 부담 스럽 시 ㄴ 분 . 3 . 총 채무 금액 은 작 으나 채무 건수 가 많 아서 관리 가 안 되 시 는 XXX 고객 님 들 께 말씀 드리 ㅂ니다 . 높 은 금리 때문 에 힘들 다고 느끼 시 었 을 때 대출 이 필요 하 시 ㄹ 때 언제 나 XXX 고객 님 들 뒤 에서 든든 하 ㄴ 파트너 로서 돕 아 드리 겠 습니다 . 당행 상품 특징 1 . 당행 거래 가 없으셔도 진행 가능 . 2 . 시중 은행 권 보다 높 은 한도 . 3 . 자체 등급 으로 판단 하 므

In [13]:
train_df['l'] = train_df['komo_morphs'].apply(lambda x: len(str(x).split(' ')))
print('mean length of data : {}'.format(train_df['l'].mean()))
print('max length of data : {}'.format(train_df['l'].max()))
print('std length of data : {}'.format(train_df['l'].std()))

mean length of data : 89.27805504401155
max length of data : 644
std length of data : 114.02346925047749


In [14]:
test_df['l'] = test_df['komo_morphs'].apply(lambda x: len(str(x).split(' ')))
print('mean length of data : {}'.format(test_df['l'].mean()))
print('max length of data : {}'.format(test_df['l'].max()))
print('std length of data : {}'.format(test_df['l'].std()))

mean length of data : 152.6691266912669
max length of data : 565
std length of data : 116.83732804508064


In [15]:
sequence_length = 220 #220 # Heuristic way to select this value (if possible, max length bewteen train and test would be fine, not considering the resources.)

In [16]:
def build_vocab(sentences, verbose=True):
    vocab = {}
    for sentence in tqdm(sentences, disable = (not verbose)):
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab
vocab = build_vocab(list(train_df['komo_morphs'].apply(lambda x: x.split())))
#vocab_length = len(vocab)

100%|██████████| 295945/295945 [00:02<00:00, 108197.51it/s]


In [17]:
len(vocab)

58290

In [18]:
%%time
max_features = len(vocab) # oov max range if the number of oov is below this value, then max_feature is set to that value.

tokenizer = Tokenizer(num_words = max_features, split=' ', oov_token='<unw>', filters=' ')
tokenizer.fit_on_texts(train_df['komo_morphs'].values)

#this takes our sentences and replaces each word with an integer
X_train = tokenizer.texts_to_sequences(train_df['komo_morphs'].values)
train_lengths = torch.from_numpy(np.array([len(x) for x in X_train]))
print("train max length: {}".format(train_lengths.max()))
maxlen = train_lengths.max()

train max length: 644
CPU times: user 22.6 s, sys: 11.1 ms, total: 22.6 s
Wall time: 22.6 s


In [19]:
%%time
X_test = tokenizer.texts_to_sequences(test_df['komo_morphs'].values)
test_lengths = torch.from_numpy(np.array([len(x) for x in X_test]))
print("test max length: {}".format(test_lengths.max()))

test max length: 565
CPU times: user 96.5 ms, sys: 21 µs, total: 96.5 ms
Wall time: 95.8 ms


In [20]:
X_train = torch.from_numpy(pad_sequences(X_train, maxlen)) #padding empty to zeros
X_test = torch.from_numpy(pad_sequences(X_test, maxlen))

In [21]:
y = torch.tensor(train_df['smishing'].values).float().unsqueeze(1)

In [22]:
X_train.shape, X_test.shape

(torch.Size([295945, 644]), torch.Size([1626, 644]))

In [23]:
print("shape of the x_train: {} * {}".format(X_train.shape[0], X_train.shape[1]))
#print("shape of the x_test: {} * {}".format(x_test_padded.shape[0], x_test_padded.shape[1]))
X_tr, X_val, y_train, y_valid = train_test_split(X_train, y, test_size=0.2)

print("shape of the validation: {} * {}".format(X_val.shape[0], X_val.shape[1]))

shape of the x_train: 295945 * 644
shape of the validation: 59189 * 644


In [24]:
word_index = tokenizer.word_index
print('Found {} unique tokens'.format(len(word_index)))

Found 57693 unique tokens


In [25]:
embeddings_index = {}
f = open(os.path.join('../KB_NLP','glove_5290.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:],dtype='float32')
    embeddings_index[word] = coefs
f.close()

print("Found {} word vectors".format(len(embeddings_index)))

Found 5290 word vectors


In [26]:
num_words = min(max_features, len(word_index))+1
print(num_words)

embedding_dim = 200 # from trained by 200 dim

#first create a matrix of zeros, this is our embedding matrix
embedding_matrix = np.zeros((num_words, embedding_dim))

for word, i in word_index.items():
    if i > max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
    else:
        embedding_matrix[i] = np.random.randn(embedding_dim)

57694


In [27]:
n_epochs = 6
cv_split = 20
n_split = 10
criterion = nn.BCEWithLogitsLoss()
batch_size = 128

In [39]:
#train_length, valid_length = lengths[trn_index], lengths[val_index]
#batch_size = 64
#train_dataset = TensorDataset(X_train, train_length, torch.tensor(y_train))
train_length = torch.from_numpy(np.array([len(x) for x in X_tr]))
valid_length = torch.from_numpy(np.array([len(x) for x in X_val]))
test_length =  torch.from_numpy(np.array([len(x) for x in X_test]))
train_dataset = TensorDataset(X_tr, train_length, torch.tensor(y_train))
valid_dataset = TensorDataset(X_val, valid_length,torch.tensor(y_valid))
test_dataset = TensorDataset(X_test, test_length)

print("num train set: ",len(train_dataset))
print("num val set: ",len(valid_dataset))
train_collator = SequenceBucketCollator(lambda length: length.max(), 
                                        sequence_index=0, 
                                        length_index=1, 
                                        label_index=2)
test_collator = SequenceBucketCollator(lambda length: length.max(), sequence_index=0, length_index=1)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=train_collator)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, collate_fn=train_collator)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=test_collator)

num train set:  236756
num val set:  59189


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [40]:
total_folds_preds1 = []
model = NeuralNet(embedding_matrix)
if use_cuda:
    model.cuda()

In [48]:
optimizer = Adam(model.parameters(),lr=0.001)
scheduler = LambdaLR(optimizer, lambda epoch: 0.6 ** epoch)
checkpoint_weights = [2 ** epoch for epoch in range(n_epochs)]
accumulation_step = 2
best_epoch = -1
best_valid_score = 0
best_valid_loss = 1
all_train_loss = []
all_valid_loss = []
total_preds = []

for epoch in range(n_epochs):
    start_time = time.time()
    
    model.train()
    train_loss = 0
    optimizer.zero_grad()
    for i, (inputs, targets) in enumerate(train_loader):
        print("i: ".format(i))
        print("inputs")
        print(inputs)
        print('targets')
        print(targets)
        targets = targets.unsqueeze(1)

        inputs = inputs.cuda()
        targest = targets.cuda()
        
        preds = model(inputs)
        loss = criterion(preds, target)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        train_loss += loss.item()/len(train_loader)
    

i: 
inputs
[tensor([[   0,    0,    0,  ...,    2,   14,   20],
        [   0,    0,    0,  ..., 1789,    2,   27],
        [   0,    0,    0,  ...,  795,    2,   27],
        ...,
        [   0,    0,    0,  ...,  102,    2,   27],
        [   0,    0,    0,  ...,    2,  101,    4],
        [   0,    0,    0,  ..., 2525,    2,   27]], dtype=torch.int32), tensor([644, 644, 644, 644, 644, 644, 644, 644, 644, 644, 644, 644, 644, 644,
        644, 644, 644, 644, 644, 644, 644, 644, 644, 644, 644, 644, 644, 644,
        644, 644, 644, 644, 644, 644, 644, 644, 644, 644, 644, 644, 644, 644,
        644, 644, 644, 644, 644, 644, 644, 644, 644, 644, 644, 644, 644, 644,
        644, 644, 644, 644, 644, 644, 644, 644, 644, 644, 644, 644, 644, 644,
        644, 644, 644, 644, 644, 644, 644, 644, 644, 644, 644, 644, 644, 644,
        644, 644, 644, 644, 644, 644, 644, 644, 644, 644, 644, 644, 644, 644,
        644, 644, 644, 644, 644, 644, 644, 644, 644, 644, 644, 644, 644, 644,
        644, 644, 

AttributeError: 'list' object has no attribute 'cuda'

In [54]:
inputs[0]

tensor([[   0,    0,    0,  ...,    2,   14,   20],
        [   0,    0,    0,  ..., 1789,    2,   27],
        [   0,    0,    0,  ...,  795,    2,   27],
        ...,
        [   0,    0,    0,  ...,  102,    2,   27],
        [   0,    0,    0,  ...,    2,  101,    4],
        [   0,    0,    0,  ..., 2525,    2,   27]], dtype=torch.int32)

In [50]:
targets.cuda()

tensor([[[0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [1.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [1.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [1.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [

In [46]:
targets = targets.unsqueeze(1)
targets

tensor([[[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[1.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],




In [ ]:
    train_loss = train_one_epoch(model, criterion, train_loader, optimizer, accumulation_step)
    val_loss, val_score = validation(model, criterion, valid_loader)
    
    elapsed = time.time() - start_time
    
    lr = [_['lr'] for _ in optimizer.param_groups]
    print("Epoch {} - train_loss: {:.6f} val_loss: {:.6f} val_score: {:.6f} lr: {:.5f} time: {:.0f}s".format(
    epoch+1, train_loss, val_loss, val_score, lr[0], elapsed))

In [ ]:
train_kwargs = dict(
    train_loader=train_loader,
    valid_loader=valid_loader,
    test_loader=test_loader,
    model=model,
    criterion=criterion,
    )

single_fold_preds, cv_score, cv_loss = train_model(n_epochs=n_epochs, accumulation_step=1, **train_kwargs)
total_folds_preds1.append(single_fold_preds)
del train_loader, valid_loader
gc.collect()
print()